In [11]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [12]:
import os
import pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [14]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Introduction to Computer Science',
  'stars': 4,
  'review': 'Dr. Johnson is an excellent professor who really cares about her students. The lectures are engaging and she is always available for extra help.'},
 {'professor': 'Professor William Lee',
  'subject': 'Organic Chemistry',
  'stars': 3,
  'review': 'The material in this course is challenging, but Professor Lee does a good job of explaining the concepts. The exams are fair but the grading can be harsh.'},
 {'professor': 'Dr. Sarah Martinez',
  'subject': 'World History',
  'stars': 5,
  'review': 'Dr. Martinez is incredibly knowledgeable and passionate about the subject matter. Her lectures are interesting and she encourages class participation. I highly recommend this course.'},
 {'professor': 'Professor David Thompson',
  'subject': 'Principles of Marketing',
  'stars': 2,
  'review': 'I was really disappointed with this course. Professor Thompson seems disorganized and often

In [16]:
from openai import OpenAI
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]

        }
    })

In [17]:
processed_data[0]

{'values': [0.011446842,
  -0.010871624,
  0.037573252,
  0.03833254,
  0.00017301485,
  -0.0018608308,
  0.036238745,
  0.043693572,
  -0.013437097,
  0.0131264785,
  0.03600866,
  -0.0006647365,
  -0.054116525,
  0.039690055,
  -0.008018541,
  0.051401496,
  -0.010670297,
  -0.012850374,
  0.016980441,
  0.04288827,
  0.026437027,
  -0.015691953,
  0.02998037,
  -0.043141365,
  -0.02367598,
  -0.041806858,
  0.019706976,
  0.00416458,
  0.04307234,
  -0.007477836,
  0.08080665,
  0.006304391,
  -0.023503415,
  -0.015496378,
  -0.014794611,
  0.008864112,
  0.012298165,
  0.017233538,
  0.010463219,
  0.009519861,
  -0.01347161,
  0.0041674557,
  -0.012907896,
  0.019683966,
  0.048732486,
  -0.0043313927,
  0.027794542,
  -0.007598632,
  0.050481148,
  0.057015628,
  -0.027403394,
  -0.0168769,
  0.03092373,
  -0.011107463,
  -0.029520197,
  -0.005067672,
  0.016600797,
  0.027932595,
  0.0093760565,
  -0.026621098,
  0.030279486,
  0.0063274,
  -0.028254718,
  -0.024320224,
  -0.017

In [18]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
    
)

{'upserted_count': 20}

In [19]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}